README - IMPORTANTTTT

Import Libraries

`python -m pip install pymongo[srv]`

additionally, please download https://dev.mysql.com/doc/index-other.html, 

Specifically, the sakila database!!! Canvas would not allow me to attach .sql files?

The other database is "sample_mflix" in MongoDB (one of the free samples that you can add to clusters)

In total, used 4 data-sets,
Northwind_DW (for dim_date),
Sample_mflix (from MongoDB),
Sakila database (from mySQL link),
Attached Top_1000_IMBD.csv


In [ ]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.39
Running PyMongo Version: 4.15.3


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [ ]:
mysql_args = {
    "uid": "root",
    "pwd" : "Passw0rd123",  #password removed for privacy
    "hostname": "localhost",
    "dbname": "sakila"   
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "patrickyin3",
    "password" : "patrickyin3",
    "cluster_name" : "ds2002",
    "cluster_subnet" : "whjs4xu",
    "cluster_location" : "atlas", # "local"
    "db_name" : "sample_mflix"  #USED SAMPLE_MFLIX IN MONGODB (free dataset)
}

mysql_args_nw = {
    "uid": "root",
    "pwd" : "Passw0rd123",  #password removed for privacy
    "hostname": "localhost",
    "dbname": "northwind_dw" 
}


Define Functions for Getting Data From and Setting Data Into Databases

In [42]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

### Load Date Dimension Data From Source Database


In [43]:
sql_date = "SELECT * FROM dim_date;"
df_dim_date = get_sql_dataframe(sql_date, **mysql_args_nw)

# Keep only relevant columns
cols_date = ['date_key', 'full_date', 'day_of_week', 'day_name_of_week',
             'day_of_month', 'day_of_year', 'weekday_weekend', 'month_of_year', 
             'calendar_year', 'calendar_quarter', 'fiscal_year', 'fiscal_quarter']

df_dim_date = df_dim_date[[c for c in cols_date if c in df_dim_date.columns]]

set_dataframe(df_dim_date, table_name='dim_date', pk_column='date_key', db_operation='insert', **mysql_args)

df_dim_date.head(5)

,date_key,full_date,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,month_of_year,calendar_year,calendar_quarter,fiscal_year,fiscal_quarter
0,20000101,2000-01-01,7,Saturday,1,1,Weekend,1,2000,1,2000,3
1,20000102,2000-01-02,1,Sunday,2,2,Weekend,1,2000,1,2000,3
2,20000103,2000-01-03,2,Monday,3,3,Weekday,1,2000,1,2000,3
3,20000104,2000-01-04,3,Tuesday,4,4,Weekday,1,2000,1,2000,3
4,20000105,2000-01-05,4,Wednesday,5,5,Weekday,1,2000,1,2000,3


### Import IMDb Top 1000 Dataset From Local CSV File


In [ ]:
#import imdb_top_1000.csv from local

import os
import pandas as pd


base_dir = os.path.dirname(os.path.abspath("__file__"))  
csv_path = os.path.join(base_dir, "imdb_top_1000.csv")

df_imdb = pd.read_csv(csv_path)
df_imdb.head(3)


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"


### Transform IMDb Dataset and Load IMDb Dimension Table


In [45]:
# Keep relevant columns and rename
cols_imdb = ['Series_Title', 'Released_Year', 'Certificate', 'Runtime', 'Genre',
             'IMDB_Rating', 'Meta_score', 'Overview', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross']
df_imdb = df_imdb[[c for c in cols_imdb if c in df_imdb.columns]]

df_imdb.rename(columns={
    'Series_Title': 'title',
    'Released_Year': 'year',
    'IMDB_Rating': 'imdb_rating',
    'Meta_score': 'metascore',
    'No_of_Votes': 'imdb_votes',
    'Gross': 'gross',
    'Star1': 'cast_1',
    'Star2': 'cast_2',
    'Star3': 'cast_3',
    'Star4': 'cast_4'
}, inplace=True)

# Add surrogate key
df_imdb.insert(0, 'movie_key', range(1, df_imdb.shape[0]+1))

# Keep final columns for MySQL
df_dim_imdb = df_imdb[['movie_key', 'title', 'year', 'Certificate', 'Runtime', 'Genre',
                        'Director', 'cast_1', 'cast_2', 'cast_3', 'cast_4', 'imdb_rating',
                        'metascore', 'imdb_votes', 'gross', 'Overview']]

# Insert into MySQL
set_dataframe(df_dim_imdb, table_name='dim_imdb_top_1000', pk_column='movie_key', db_operation='insert', **mysql_args)

# Verify
df_imdb_mysql = get_sql_dataframe("SELECT * FROM dim_imdb_top_1000 LIMIT 5;", **mysql_args)
df_imdb_mysql.head(5)

,movie_key,title,year,Certificate,Runtime,Genre,Director,cast_1,cast_2,cast_3,cast_4,imdb_rating,metascore,imdb_votes,gross,Overview
0,1,The Shawshank Redemption,1994,A,142 min,Drama,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,9.3,80.0,2343110,"28,341,469",Two imprisoned men bond over a number of years...
1,2,The Godfather,1972,A,175 min,"Crime, Drama",Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,9.2,100.0,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...
2,3,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,9.0,84.0,2303232,"534,858,444",When the menace known as the Joker wreaks havo...
3,4,The Godfather: Part II,1974,A,202 min,"Crime, Drama",Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,9.0,90.0,1129952,"57,300,000",The early life and career of Vito Corleone in ...
4,5,12 Angry Men,1957,U,96 min,"Crime, Drama",Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,9.0,96.0,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...


### Transform MongoDB Movie Data and Load Movies Dimension Table


In [ ]:
import numpy as np
import pandas as pd

# DIM MOVIES (MongoDB)

client = get_mongo_client(**mongodb_args)
df_movies = get_mongo_dataframe(client, mongodb_args["db_name"], "embedded_movies", {})
client.close()

# Rename primary key and keep relevant columns
df_movies.rename(columns={'_id': 'movie_id'}, inplace=True)
cols_movies = ['movie_id', 'title', 'year', 'runtime', 'genres', 'directors', 'cast', 'metacritic']
df_movies = df_movies[[c for c in cols_movies if c in df_movies.columns]]

# Convert list columns to comma-separated strings
for col in ['genres', 'directors', 'cast']:
    if col in df_movies.columns:
        df_movies[col] = df_movies[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Add surrogate key
df_movies.insert(0, 'movie_key', range(1, df_movies.shape[0]+1))

# Keep only dimension columns
df_dim_movies = df_movies[['movie_key', 'title', 'year', 'runtime', 'genres', 'directors', 'cast', 'metacritic']]

# Insert into MySQL
set_dataframe(df_dim_movies, table_name='dim_movies', pk_column='movie_key', db_operation='insert', **mysql_args)

# Verify
df_movies_mysql = get_sql_dataframe("SELECT * FROM dim_movies LIMIT 5;", **mysql_args)
df_movies_mysql.head(5)


,movie_key,title,year,runtime,genres,directors,cast,metacritic
0,1,Scarface,1932,93.0,"Action, Crime, Drama","Howard Hawks, Richard Rosson","Paul Muni, Ann Dvorak, Karen Morley, Osgood Pe...",None
1,2,The Charge of the Light Brigade,1936,115.0,"Action, Adventure, Romance",Michael Curtiz,"Errol Flynn, Olivia de Havilland, Patric Knowl...",None
2,3,Flash Gordon,1936,245.0,"Action, Adventure, Sci-Fi","Frederick Stephani, Ray Taylor","Buster Crabbe, Jean Rogers, Charles Middleton,...",None
3,4,The Perils of Pauline,1914,199.0,Action,"Louis J. Gasnier, Donald MacKenzie","Pearl White, Crane Wilbur, Paul Panzer, Edward...",None
4,5,From Hand to Mouth,1919,22.0,"Comedy, Short, Action","Alfred J. Goulding, Hal Roach","Harold Lloyd, Mildred Davis, 'Snub' Pollard, P...",None


In [ ]:
# DIM USERS (MongoDB)

client = get_mongo_client(**mongodb_args)
df_users = get_mongo_dataframe(client, mongodb_args["db_name"], "users", {})
client.close()

# Rename primary key and keep relevant columns
df_users.rename(columns={'_id': 'user_id'}, inplace=True)
cols_users = ['user_id', 'name', 'email']
df_users = df_users[[c for c in cols_users if c in df_users.columns]]

# Add surrogate key
df_users.insert(0, 'user_key', range(1, df_users.shape[0]+1))

# Keep only dimension columns
df_dim_users = df_users[['user_key', 'name', 'email']]

# Insert into MySQL
set_dataframe(df_dim_users, 'dim_users', 'user_key', 'insert', **mysql_args)

# Verify
df_users_mysql = get_sql_dataframe("SELECT * FROM dim_users LIMIT 5;", **mysql_args)
df_users_mysql.head(5)

,user_key,name,email
0,1,Ned Stark,sean_bean@gameofthron.es
1,2,Robert Baratheon,mark_addy@gameofthron.es
2,3,Jaime Lannister,nikolaj_coster-waldau@gameofthron.es
3,4,Catelyn Stark,michelle_fairley@gameofthron.es
4,5,Cersei Lannister,lena_headey@gameofthron.es


In [ ]:
# FACT COMMENTS (MongoDB)

client = get_mongo_client(**mongodb_args)
df_comments = get_mongo_dataframe(client, mongodb_args["db_name"], "comments", {})
client.close()

# Rename primary key and keep relevant columns
df_comments.rename(columns={'_id': 'comment_id'}, inplace=True)
cols_comments = ['comment_id', 'text', 'date', 'movie_id', 'user_id']
df_comments = df_comments[[c for c in cols_comments if c in df_comments.columns]]

# Add surrogate key
df_comments.insert(0, 'comment_key', range(1, df_comments.shape[0]+1))

# Map user_key and movie_key 
df_comments['user_key'] = np.random.choice(df_dim_users['user_key'], size=len(df_comments))
df_comments['movie_key'] = np.random.choice(df_dim_movies['movie_key'], size=len(df_comments))

# Keep only fact table columns
df_fact_comments = df_comments[['comment_key', 'movie_key', 'user_key', 'text', 'date']]

# Insert into MySQL
set_dataframe(df_fact_comments, 'fact_comments', 'comment_key', 'insert', **mysql_args)

# Verify
df_fact_comments_mysql = get_sql_dataframe("SELECT * FROM fact_comments LIMIT 5;", **mysql_args)
df_fact_comments_mysql.head(5)



,comment_key,movie_key,user_key,text,date
0,1,823,170,Eius veritatis vero facilis quaerat fuga tempo...,2002-08-18 04:56:07
1,2,2086,14,Id error ab at molestias dolorum incidunt. Non...,1975-01-21 00:31:22
2,3,3306,43,Minima odit officiis minima nam. Aspernatur id...,1981-11-08 04:32:25
3,4,529,144,Iure laboriosam quo et necessitatibus sed. Id ...,1970-11-15 05:54:02
4,5,1075,108,Nobis incidunt ea tempore cupiditate sint. Ita...,2012-11-26 11:00:57


In [ ]:
# SAKILA ETL (MySQL)

#DIM ACTOR
df_actor = get_sql_dataframe("SELECT * FROM actor;", **mysql_args)
df_actor.insert(0, 'actor_key', range(1, len(df_actor)+1))
df_dim_actor = df_actor[['actor_key', 'first_name', 'last_name']]
set_dataframe(df_dim_actor, 'dim_actor', 'actor_key', 'insert', **mysql_args)

#DIM FILM
df_film = get_sql_dataframe("SELECT * FROM film;", **mysql_args)
df_film.insert(0, 'film_key', range(1, len(df_film)+1))
df_dim_film = df_film[['film_key', 'title', 'release_year', 'rating', 'length']]
set_dataframe(df_dim_film, 'dim_film', 'film_key', 'insert', **mysql_args)

#FACT FILM_ACTOR
df_film_actor = get_sql_dataframe("SELECT * FROM film_actor;", **mysql_args)
film_map = dict(zip(df_film['film_id'], df_dim_film['film_key']))
actor_map = dict(zip(df_actor['actor_id'], df_dim_actor['actor_key']))

df_film_actor['film_key'] = df_film_actor['film_id'].map(film_map)
df_film_actor['actor_key'] = df_film_actor['actor_id'].map(actor_map)

df_fact_film_actor = df_film_actor[['film_key', 'actor_key']]
df_fact_film_actor.insert(0, 'film_actor_key', range(1, len(df_fact_film_actor)+1))

set_dataframe(df_fact_film_actor, 'fact_film_actor', 'film_actor_key', 'insert', **mysql_args)

# Verify
df_fact_film_actor_mysql = get_sql_dataframe("SELECT * FROM fact_film_actor LIMIT 5;", **mysql_args)
df_fact_film_actor_mysql.head(5)

,film_actor_key,film_key,actor_key
0,1,1,1
1,2,23,1
2,3,25,1
3,4,106,1
4,5,140,1


In [ ]:
# QUERY 1: Total Comments per User per Movie 
# This query joins fact_comments, dim_users, and dim_movies to find how many comments each user has made per movie. It shows the top 20 user–movie combinations with the most comments.

sql_comments_per_user_movie = """
SELECT
    u.user_key,
    u.name AS user_name,
    m.movie_key,
    m.title AS movie_title,
    COUNT(c.comment_key) AS total_comments
FROM
    fact_comments c
JOIN
    dim_users u ON c.user_key = u.user_key
JOIN
    dim_movies m ON c.movie_key = m.movie_key
GROUP BY
    u.user_key, u.name, m.movie_key, m.title
ORDER BY
    total_comments DESC
LIMIT 20;
"""

df_comments_per_user_movie = get_sql_dataframe(sql_comments_per_user_movie, **mysql_args)
df_comments_per_user_movie.head(5)


,user_key,user_name,movie_key,movie_title,total_comments
0,158,Thomas Green,1620,Black Knight,3
1,174,Nicholas Johnson,1356,Resident Evil,3
2,65,Othell Yarwyck,663,RED,3
3,32,Gendry,863,The Last Man,3
4,181,Shawn Mccormick,1983,Outbound,3


In [ ]:
#QUERY 2: Average IMDb Rating per User
# This query computes the average IMDb rating of all movies commented on by each user. It joins fact_comments, dim_users, and dim_imdb_top_1000 and returns the top 20 users ranked by their average movie rating.

sql_avg_rating_per_user = """
SELECT
    u.user_key,
    u.name AS user_name,
    COUNT(c.comment_key) AS total_comments,
    AVG(m.imdb_rating) AS avg_imdb_rating
FROM
    fact_comments c
JOIN
    dim_users u ON c.user_key = u.user_key
JOIN
    dim_imdb_top_1000 m ON c.movie_key = m.movie_key
GROUP BY
    u.user_key, u.name
ORDER BY
    avg_imdb_rating DESC
LIMIT 20;
"""

df_avg_rating_per_user = get_sql_dataframe(sql_avg_rating_per_user, **mysql_args)
df_avg_rating_per_user.head(5)


,user_key,user_name,total_comments,avg_imdb_rating
0,167,Mary Mitchell,57,8.050877
1,6,Daenerys Targaryen,60,8.048333
2,45,Meryn Trant,63,8.044444
3,83,Bowen Marsh,58,8.027586
4,32,Gendry,68,8.020588


In [ ]:
#QUERY 3: Comments and Average Ratings by Sakila Customers
# This query integrates the Sakila database by linking users to Sakila customers (matching by user_key). It shows, for each customer, the total number of comments and the average IMDb rating of the movies they commented on.

sql_comments_sakila = """
SELECT
    cust.customer_id,
    cust.first_name,
    cust.last_name,
    COUNT(c.comment_key) AS total_comments,
    AVG(m.imdb_rating) AS avg_movie_rating
FROM
    fact_comments c
JOIN
    dim_users u ON c.user_key = u.user_key
JOIN
    dim_imdb_top_1000 m ON c.movie_key = m.movie_key
JOIN
    sakila.customer cust ON cust.customer_id = u.user_key
GROUP BY
    cust.customer_id, cust.first_name, cust.last_name
ORDER BY
    total_comments DESC
LIMIT 20;
"""

df_comments_sakila = get_sql_dataframe(sql_comments_sakila, **mysql_args)
df_comments_sakila.head(5)


,customer_id,first_name,last_name,total_comments,avg_movie_rating
0,55,DORIS,REED,87,8.003448
1,53,HEATHER,MORRIS,83,7.892771
2,7,MARIA,MILLER,81,7.940741
3,74,DENISE,KELLY,81,7.972840
4,165,LORRAINE,STEPHENS,80,7.960000
